In [37]:
from copy import deepcopy
from itertools import permutations

def pow_set(list):
    x = len(list)
    masks = [1 << i for i in range(x)]
    for i in range(1 << x):
        yield [s_set for mask, s_set in zip(masks, list) if i & mask]

def mat_to_graph(mat):
    keys = ["Start"]
    for num in range(1, len(mat)-1):
        keys.append(num-1)
    keys.append("Bulkhead")
    graph = dict(zip(keys, mat))
    return graph


def make(graph, source):
    dist = {}
    pre = {}
    for node in graph:
        dist[node] = 1000
        pre[node] = None
    dist[source] = 0 
    return dist, pre


def wait(node, neighbour, graph, dist, pre):
    id = get_neighbour(neighbour, len(graph))
    if dist[node] + graph[node][id] < dist[neighbour]:
        dist[neighbour] = dist[node] + graph[node][id]
        pre[neighbour] = node


def get_neighbour(neighbour, g_size):
    if neighbour == "Bulkhead":
        return g_size - 1
    elif neighbour == "Start":
        return 0
    else:
        return int(neighbour)+1
        
def bell_ford_alg(mat, graph, time_limit, source):
    dist, pred = make(graph, source)
    for num in range(len(graph)-1):
        for node in graph:
            temp = dict(graph)
            del temp[node]
            for neighbour in temp:
                wait(node, neighbour, graph, dist, pred)

    for node in graph:
        for neighbour in graph:
            id = get_neighbour(neighbour, len(graph))
            if dist[node] + graph[node][id] < dist[neighbour]:
                return [num for num in range(0, len(graph)-2)]

    paths = floyd_alg(mat)
    return save_max_bunnies(mat, paths, time_limit)


def floyd_alg(mat):
    n = len(mat)
    paths = deepcopy(mat) 
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if paths[i][k] + paths[k][j] < paths[i][j]:
                    paths[i][j] = paths[i][k] + paths[k][j]
    return paths


def save_max_bunnies(mat, paths, time_limit):
    n = len(mat)-2
    bunnies = []
    for num in range(n):
        bunnies.append(num)
    p_set = pow_set(bunnies)
    p_set = sorted(p_set)

    
    saved_bunnies = []
    for s_set in p_set:
        for permutation in permutations(s_set):
            sub_sum = 0
            prev = 0
            next = len(mat)-1
            
            for bunnyid in permutation:
                next = bunnyid+1
                sub_sum += paths[prev][next]
                prev = next
            sub_sum += paths[prev][len(mat)-1]
            if sub_sum <= time_limit and len(s_set) > len(saved_bunnies):
                saved_bunnies = s_set
                if len(saved_bunnies) == n:
                    break
            else:
                continue
    return saved_bunnies


def solution(times, time_limit):
    if len(times) <= 2:
        return []
    
    graph = mat_to_graph(times)
    return bell_ford_alg(times, graph, time_limit, "Start")

In [38]:
answer([[0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]], 3)

[0, 1]

In [ ]:
wait